In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
IMG_SIZE = (128, 128)
BATCH_SIZE = 16
DATASET_DIR = "images"

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_DIR,
    labels='inferred',
    label_mode='int',
    class_names=['spiral', 'elliptical'],
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_DIR,
    labels='inferred',
    label_mode='int',
    class_names=['spiral', 'elliptical'],
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

class_names = ['spiral', 'elliptical']
print("Classes:", class_names)

In [ ]:
# Optimize dataset
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(100).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:

# Build CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(128, 128, 3)),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

In [ ]:
# Compile model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Train model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

In [ ]:
final_train_loss, final_train_acc = model.evaluate(train_ds)
final_val_loss, final_val_acc = model.evaluate(val_ds)

print(f"Final Training Accuracy: {final_train_acc * 100:.2f}%")
print(f"Final Validation Accuracy: {final_val_acc * 100:.2f}%")

In [ ]:
def predict_image(image_path, threshold=0.6):
    img = load_img(image_path, target_size=IMG_SIZE)
    img_array = img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch axis
    predictions = model.predict(img_array)[0]

    # Get top prediction and confidence
    top_index = np.argmax(predictions)
    top_confidence = predictions[top_index]
    predicted_class = class_names[top_index]

    # Fallback to irregular if not confident
    if top_confidence < threshold:
        predicted_class = 'irregular'

    print(f"Prediction: {predicted_class} (Confidence: {top_confidence:.2f})")
    return predicted_class

In [ ]:
predict_image("988001.jpg")

In [ ]:
predict_image("987261.jpg")

In [ ]:
predict_image("images (2).jpg")

In [ ]:
predict_image("image0.jpg")

In [ ]:
predict_image("M104_ngc4594_sombrero_galaxy_hi-res.jpg")